In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

plt.rcParams['animation.embed_limit'] = 2**128


def der(y, t, k, m, left_force = None, right_force = None):
    N = y.size
    N //= 2
    derivative = np.zeros_like(y)

    if left_force is not None:
        derivative[0] = left_force(y, t)
    if right_force is not None:
        derivative[N - 1] = right_force(y, t)

    derivative[1:N - 1] = (y[N + 2:2*N] + y[N:2*N - 2] - 2 * y[N+1:2 * N - 1]) * k / m
    derivative[N:2 * N] = y[0:N]

    return derivative


k = 25
m = 0.25
N = 100

left_force = None
right_force = None
left_pos = None
right_right = None

initial_condition = np.zeros(2 * N)

dt = 0.001
max_time = 20
fps = 30
speedup = 1
frame_select = int(speedup / (fps * dt))

times = np.arange(0, max_time, dt)

frequency = 0.2
omega = 2 * np.pi * frequency
A = 0.8


def left_force(y, t):
    N = y.size
    N //= 2
    return -omega**2 * y[N]


initial_condition[0] = omega * A

result = odeint(der, initial_condition, times, (k, m, left_force, right_force))
frames = len(result)
result = result[::frame_select, :]
selected_frames = len(result)

frames, selected_frames

fig, ax = plt.subplots(figsize = (12, 6))
ax.axes.get_xaxis().set_visible(False)
ax.axes.get_yaxis().set_visible(False)
fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=None, hspace=None)

xs = np.linspace(0, 1, N)

plot, = ax.plot(xs, initial_condition[N:], 'o-', color='red')
ax.set_ylim(-1, 1)
plt.close()


def frame(i):
    plot.set_data(xs, result[i, N:])
    return plot,


anim = FuncAnimation(fig, frame, frames=len(result), interval=1000 / fps, blit=True)

HTML(anim.to_jshtml())
